In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
def check_data(df: pd.DataFrame) -> None:
    # Смотрим пропуски
    print(' Пропуски '.center(30, '#'))
    loss_count = df.isna().sum().sum()
    if loss_count == 0:
        print('Пропусков нет')
    else:
        print(df.isna().sum() * 100 / df.shape[0])
    
    # Распределение данных
    print()
    print(' Распределение '.center(30, '#'))
    print(df.describe())

    # Уникальные значения для качественных данных
    print()
    print(' Уникальные значения '.center(30, '#'))
    for column in df.columns:
        if df[column].dtype == 'object':
            uniq_value = df[column].nunique()
            if uniq_value <= 15:
                print(f'Уникальных значений в столбце {column} {uniq_value}: {", ".join(df[column].unique().tolist())}')
            else:
                print(f'Уникальных значений в столбце {column} более 15: {", ".join(df[column].unique().tolist()[:15])}')

# Отберем из всех муниципальных образований Сахалинскую область

In [3]:
DATA_DIR = '../data'
TRANSFORM_DIR = './transform_data'
FINAL_DIR = '../final_data'

# Исключаем Курильские острова
# чтобы анализировать именно остров Сахалин
EXCLUDE_TERRITORIES = [
    'городской округ Курильский', 
    'городской округ Северо-Курильский', 
    'городской округ Южно-Курильский']

In [5]:
# Читаем данные по муниципалитетам
districts = pd.read_excel(f'{DATA_DIR}/districts.xlsx')

# Отберем муниципалитеты Сахалинской области (актуальные)
districts = districts[(districts['region_name'] == 'Сахалинская область') & 
                      (districts['year_to'] == 9999)]
print(f'Найдено {districts.shape[0]} записей по Сахалинской области')
print(f'Уникальных муниципальных образований: {districts["municipal_district_name"].nunique()}')

# Для быстрой фильтрации в дальнейшем сохраним id территорий
t_id = districts['territory_id'].unique()
print(f"\nID территорий для анализа: {t_id}")

# Выберем нужные столбцы
districts = districts[[
    'territory_id', 
    'municipal_district_name', 
    'municipal_district_center', 
    'municipal_district_center_lat', 
    'municipal_district_center_lon'
]].copy()

# Сохраним данные по отобранным муниципалитетам
districts.to_csv(f'{TRANSFORM_DIR}/districts.csv', index=False)

districts_dfo = pd.read_excel(f'{DATA_DIR}/district_name.xlsx')
territory_id_dfo = districts_dfo.loc[
    districts_dfo['federal_districts_name_short'] == 'ДФО', 
    'territory_id'
]

print(f"Найдено {len(territory_id_dfo)} территорий в ДФО")

Найдено 18 записей по Сахалинской области
Уникальных муниципальных образований: 18

ID территорий для анализа: [1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966
 1967 1968 1969 1970]
Найдено 277 территорий в ДФО


## Население

In [6]:
# Загружаем и фильтруем данные
population = pd.read_parquet(f'{DATA_DIR}/population.parquet')
population = population[population['territory_id'].isin(t_id)]

check_data(population)

# Восстановление данных по Южно-Сахалинску
population_region = pd.read_excel(f'{DATA_DIR}/population_all.xlsx')
population_region['age'] = population_region['age'].astype(str)

# Берем данные за 2024 год
population_2024 = population[population['year'] == 2024].copy()

# Суммируем население по возрастно-половым группам для всех муниципалитетов кроме Южно-Сахалинска
population_wo_center = population_2024.pivot_table(
    index=['age', 'gender'],
    values='value',
    aggfunc='sum'
).reset_index()


population_lost = population_region.merge(
    population_wo_center,
    how='left',
    on=['age', 'gender'],
    suffixes=('_region', '_municipalities')
)

population_lost['value_municipalities'] = population_lost['value_municipalities'].fillna(0)
population_lost['value_diff'] = population_lost['value_region'] - population_lost['value_municipalities']

# Проверяем на отрицательные значения
negative_count = (population_lost['value_diff'] < 0).sum()
if negative_count > 0:
    print(f"ВНИМАНИЕ: Найдено {negative_count} записей с отрицательной разницей населения")
    # Можно посмотреть проблемные записи
    print(population_lost[population_lost['value_diff'] < 0].head())


population_lost['territory_id'] = 1953
population_lost = population_lost[['territory_id', 'age', 'gender', 'value_diff']].rename(
    columns={'value_diff': 'value'}
)

# Объединяем все данные
population_full = pd.concat([
    population_2024[['territory_id', 'age', 'gender', 'value']],
    population_lost
], ignore_index=True)

########## Пропуски ##########
Пропусков нет

####### Распределение ########
       territory_id         year         value
count    4896.00000  4896.000000   4896.000000
mean     1962.00000  2023.500000    222.115605
std         4.89948     0.500051   1084.575547
min      1954.00000  2023.000000      3.000000
25%      1958.00000  2023.000000     54.000000
50%      1962.00000  2023.500000     89.000000
75%      1966.00000  2024.000000    141.000000
max      1970.00000  2024.000000  21280.000000

#### Уникальные значения #####
Уникальных значений в столбце period 1: год
Уникальных значений в столбце age более 15: 0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21
Уникальных значений в столбце gender 2: Женщины, Мужчины


In [ ]:
population_lost

,territory_id,age,gender,value
0,1953,0,Мужчины,1146.0
1,1953,1,Мужчины,1270.0
2,1953,2,Мужчины,630.0
3,1953,3,Мужчины,471.0
4,1953,4,Мужчины,621.0
...,...,...,...,...
139,1953,67,Женщины,1236.0
140,1953,68,Женщины,1277.0
141,1953,69,Женщины,1277.0
142,1953,70+,Женщины,12662.0


In [7]:
# После востановления численности населения Южно-Сахалинска, уберем данные по Курильским островам, 
# так как целью работы является анализ транспорта и цены жизни в границах одного острова
districts = districts[~districts['municipal_district_name'].isin(EXCLUDE_TERRITORIES)]

print(f'Осталось {districts["territory_id"].nunique()} муниципалитетов')

# Обновим t_id и сохраним измененный список муниципалитетов
t_id = districts['territory_id'].unique().tolist()

districts.to_csv(f'{TRANSFORM_DIR}/districts.csv', index=False)

# Обновим ранее полученный датасет по населению
population_full = population_full[population_full['territory_id'].isin(t_id)].copy()

Осталось 15 муниципалитетов


In [ ]:
population_full.loc[population_full['age'] == 'Всего', 'value'].sum() == 436776

True

In [ ]:
# Добавим названия муниципальных образований
# и сохраним данные для построения демографической пирамиды 
population_triangle = population_full.merge(districts, how='left', on='territory_id')

population_triangle.to_csv(f'{FINAL_DIR}/population_triangle.csv')

In [ ]:
# Создаем возрастные группы
def create_age_groups(age_str: str) -> str:
    if age_str == 'Всего':
        return 'Всего'
    elif age_str == '70+':
        return '65+'  # Объединяем с старшей группой
    else:
        try:
            age = int(age_str)
            if age < 0:
                return 'Всего'
            elif age <= 14:
                return '0-14'
            elif age <= 24:
                return '15-24'
            elif age <= 44:
                return '25-44'
            elif age <= 64:
                return '45-64'
            else:
                return '65+'
        except ValueError:
            return 'Всего'


In [ ]:
# Применяем функцию
population_full['age_group'] = population_full['age'].apply(create_age_groups)

# Агрегируем по возрастным группам
population_age_groups = population_full.pivot_table(
    index=['territory_id', 'age_group', 'gender'], 
    values='value', 
    aggfunc='sum'
).reset_index()

# Сохраняем данные с возрастной структурой
population_age_groups.to_csv(f'{FINAL_DIR}/population_with_age_groups.csv', index=False)

In [ ]:
# Сохраняем общую численность
territory_population_total = population_age_groups[
    population_age_groups['age_group'] == 'Всего'
].pivot_table(
    index='territory_id',
    values='value',
    aggfunc='sum'
).reset_index()

territory_population_total['value'] = territory_population_total['value'].astype(int)

territory_population_total.to_csv(f'{FINAL_DIR}/territory_population.csv', index=False)

## Зарплаты

In [8]:
salary = pd.read_parquet('../data/salary.parquet')

check_data(salary)

# Возбмем нужные столбцы за годовой период
salary = salary.loc[
    salary['period'] == 'январь-декабрь', 
    ['territory_id', 'year', 'okved_name', 'okved_letter', 'value']]

salary_local = salary[salary['territory_id'].isin(t_id)].copy()
salary_dfo = salary[salary['territory_id'].isin(territory_id_dfo)].copy()

# Анализируем выбросы
def analyze_outliers(df, name):
    Q1 = df['value'].quantile(0.25)
    Q3 = df['value'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df['value'] < lower_bound) | (df['value'] > upper_bound)]
    print(f"{name}: {len(outliers)} выбросов ({len(outliers)/len(df)*100:.1f}%)")
    return lower_bound, upper_bound

print("\nАнализ выбросов:")
lower_local, upper_local = analyze_outliers(salary_local, "Сахалин")
lower_dfo, upper_dfo = analyze_outliers(salary_dfo, "ДФО")

# Агрегируем данные с учетом количества наблюдений
def safe_aggregate(df, group_cols, value_col='value'):
    """Безопасная агрегация с проверкой количества наблюдений"""
    aggregated = df.groupby(group_cols).agg({
        value_col: ['median', 'count', 'mean', 'std']
    }).round(2)
    
    # Выравниваем колонки
    aggregated.columns = ['_'.join(col).strip() for col in aggregated.columns.values]
    aggregated = aggregated.reset_index()
    
    # Добавляем флаг надежности
    aggregated['reliable'] = aggregated[value_col + '_count'] >= 3
    return aggregated

# Агрегируем по ДФО
salary_dfo_median = safe_aggregate(salary_dfo, ['year', 'okved_name'])
print(f"ДФО: {len(salary_dfo_median)} комбинаций год-отрасль")
print(f"Из них надежных (n>=3): {salary_dfo_median['reliable'].sum()}")

# Отбираем зарплаты по Сахалину
salary_median = safe_aggregate(salary_local, ['year', 'okved_name'])
print(f"\nСахалин: {len(salary_median)} комбинаций год-отрасль")
print(f"Из них надежных (n>=3): {salary_median['reliable'].sum()}")

# Объединяем данные
salary_diff = salary_median.merge(
    salary_dfo_median, 
    on=['year', 'okved_name'], 
    how='inner',  # Только общие отрасли
    suffixes=('_local', '_dfo')
)

print(f"\nПосле объединения: {len(salary_diff)} общих комбинаций год-отрасль")

# Вычисляем разницу только для надежных данных
reliable_mask = salary_diff['reliable_local'] & salary_diff['reliable_dfo']
print(f"Надежных сравнений: {reliable_mask.sum()}")

salary_diff['diff_absolute'] = salary_diff['value_median_local'] - salary_diff['value_median_dfo']
salary_diff['diff_relative'] = (salary_diff['diff_absolute'] / salary_diff['value_median_dfo']) * 100

salary_2024 = salary_diff[salary_diff['year'] == 2024]

# Показываем топ-5 отраслей с наибольшей разницей
print("\nТоп-5 отраслей с наибольшей надбавкой на Сахалине:")
top_pos = salary_2024.nlargest(5, 'diff_relative')[['okved_name', 'value_median_local', 'value_median_dfo', 'diff_relative']]
print(top_pos.to_string(index=False))

print("\nТоп-5 отраслей с наименьшей надбавкой (или с отрицательной):")
top_neg = salary_2024.nsmallest(5, 'diff_relative')[['okved_name', 'value_median_local', 'value_median_dfo', 'diff_relative']]
print(top_neg.to_string(index=False))

# Сохраняем полные данные с метаинформацией
salary_diff.to_csv(f'{FINAL_DIR}/salary_diff_detailed.csv', index=False)

# Сохраняем упрощенную версию для визуализации
salary_diff_simple = salary_diff[[
    'year', 
    'okved_name', 
    'value_median_local', 
    'value_median_dfo', 
    'diff_relative', 
    'reliable_local', 
    'reliable_dfo']]
salary_diff_simple.to_csv(f'{FINAL_DIR}/salary_diff.csv', index=False)

########## Пропуски ##########
territory_id     0.000000
year             0.000000
period           0.000000
okved_name       0.000000
okved_letter     0.000000
value           19.399046
dtype: float64

####### Распределение ########
        territory_id           year         value
count  369812.000000  369812.000000  2.980720e+05
mean     1318.855794    2023.493410  6.109539e+04
std       770.225466       0.499957  6.540297e+04
min         1.000000    2023.000000  1.250000e+02
25%       651.000000    2023.000000  3.753828e+04
50%      1303.000000    2023.000000  4.861255e+04
75%      1961.000000    2024.000000  6.920465e+04
max      3101.000000    2024.000000  1.500000e+07

#### Уникальные значения #####
Уникальных значений в столбце period 4: январь-декабрь, январь-июнь, январь-март, январь-сентябрь
Уникальных значений в столбце okved_name более 15: Все отрасли, Обрабатывающие производства, Услуги ЖКХ, Строительство, Торговля, ИТ и связь, Финансы и страхование, Операции с недвижимос

In [17]:
salary_2024['diff_relative'].median()

25.46884113763094

## Траты

In [18]:
expenses = pd.read_parquet(f'{DATA_DIR}/consumption.parquet')
expenses['date'] = pd.to_datetime(expenses['date'])

check_data(expenses)

expenses_2024 = expenses[(expenses['date'] >= '2024-01-01') ]

# Анализируем выбросы в расходах
def analyze_expense_outliers(df, name):
    Q1 = df['value'].quantile(0.25)
    Q3 = df['value'].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[df['value'] > upper_bound]
    print(f"{name}: {len(outliers)} выбросов (> {upper_bound:,.0f} руб.)")
    return upper_bound

print("\nАнализ выбросов в расходах:")
upper_expenses = analyze_expense_outliers(expenses_2024, "Все данные 2024")

# Создаем отдельные наборы данных
expenses_dfo = expenses_2024[expenses_2024['territory_id'].isin(territory_id_dfo)]
expenses_local = expenses_2024[expenses_2024['territory_id'].isin(t_id)]

# Перезагружаем или фильтруем актуальные данные о зарплатах
salary = pd.read_parquet('../data/salary.parquet')
salary_2024 = salary[
    (salary['period'] == 'январь-декабрь') & 
    (salary['year'] == 2024) &
    (salary['okved_name'] == 'Все отрасли')
]

# Медианные зарплаты по регионам
salary_dfo_2024 = salary_2024[salary_2024['territory_id'].isin(territory_id_dfo)]
salary_local_2024 = salary_2024[salary_2024['territory_id'].isin(t_id)]

median_salary_dfo = salary_dfo_2024['value'].median()
median_salary_local = salary_local_2024['value'].median()

# Анализ расходов по ДФО
expenses_dfo_median = expenses_dfo.groupby('category')['value'].agg([
    'median'
]).round(2).reset_index()

expenses_dfo_median['salary'] = median_salary_dfo
expenses_dfo_median['consumption_percent'] = round(
    expenses_dfo_median['median'] * 100 / expenses_dfo_median['salary'], 
    2
)

# Анализ расходов по Сахалину с детализацией по муниципалитетам
expenses_local_detailed = expenses_local.merge(
    salary_local_2024[['territory_id', 'value']].rename(columns={'value': 'salary'}),
    on='territory_id',
    how='left'
)

# Проверяем пропуски в зарплатах
missing_salary = expenses_local_detailed['salary'].isna().sum()
if missing_salary > 0:
    print(f"ВНИМАНИЕ: {missing_salary} записей без данных о зарплате")

# Считаем процент расходов для каждой записи
expenses_local_detailed['consumption_percent'] = round(
    expenses_local_detailed['value'] * 100 / expenses_local_detailed['salary'], 
    2
)

# Агрегируем по категориям (учитываем вариативность между муниципалитетами)
expenses_local_median = expenses_local_detailed.groupby('category').agg({
    'consumption_percent': 'median',
    'value': 'median',
    'salary': 'median'
}).round(2)

print(f"Расходы по Сахалину: {len(expenses_local_median)} категорий")

# Сравниваем разницу в долях расходов
expenses_diff = expenses_local_median.merge(
    expenses_dfo_median, 
    on='category', 
    how='inner',  # Только общие категории
    suffixes=('_local', '_dfo')
)

print(f"\n=== РЕЗУЛЬТАТЫ СРАВНЕНИЯ ===")
print(f"Сравниваем {len(expenses_diff)} категорий")

expenses_diff.rename(columns={
    'value': 'value_local',
    'median': 'value_dfo'
}, inplace=True)

# Сохраняем данные
expenses_diff.to_csv(f'{FINAL_DIR}/consumption_diff.csv', index=False)

print("\nДанные сохранены")


########## Пропуски ##########
Пропусков нет

####### Распределение ########
                                date   territory_id          value
count                         303126  303126.000000  303126.000000
mean   2023-12-14 20:03:44.318599936    1394.997071    8109.852213
min              2023-01-01 00:00:00       1.000000      10.000000
25%              2023-06-01 00:00:00     734.000000    1161.000000
50%              2023-12-01 00:00:00    1429.000000    2799.500000
75%              2024-06-01 00:00:00    2070.000000   11498.000000
max              2024-12-01 00:00:00    3101.000000   91003.000000
std                              NaN     775.252241   10929.969941

#### Уникальные значения #####
Уникальных значений в столбце category 6: Продовольствие, Здоровье, Маркетплейсы, Общественное питание, Транспорт, Все категории

Анализ выбросов в расходах:
Все данные 2024: 10962 выбросов (> 28,338 руб.)
Расходы по Сахалину: 6 категорий

=== РЕЗУЛЬТАТЫ СРАВНЕНИЯ ===
Сравниваем 6 катего

In [ ]:
expenses_diff

,category,consumption_percent_local,value_local,salary_local,value_dfo,salary_dfo,consumption_percent_dfo
0,Все категории,32.78,36083.5,107272.0,33826.0,94017.1,35.98
1,Здоровье,1.15,1278.5,107272.0,1316.0,94017.1,1.40
2,Маркетплейсы,2.99,3431.5,107272.0,3355.0,94017.1,3.57
3,Общественное питание,0.91,984.5,107272.0,1032.0,94017.1,1.10
4,Продовольствие,16.08,17652.0,107272.0,15221.0,94017.1,16.19
5,Транспорт,1.65,1826.5,107272.0,1806.0,94017.1,1.92


In [ ]:
expenses_local_detailed = expenses_local_detailed.pivot_table(
    index=['territory_id', 'category'],
    values=['value', 'salary'],
    aggfunc='median'
).reset_index()

expenses_local_detailed['consumption_percent'] = round(
    expenses_local_detailed['value'] * 100 / expenses_local_detailed['salary'],
    2)

expenses_local_detailed[expenses_local_detailed['category'] == 'Все категории'].to_csv(f'{FINAL_DIR}/consumption_all.csv')
expenses_local_detailed[expenses_local_detailed['category'] == 'Продовольствие'].to_csv(f'{FINAL_DIR}/consumption_prod.csv')
expenses_local_detailed[expenses_local_detailed['category'] == 'Транспорт'].to_csv(f'{FINAL_DIR}/consumption_transport.csv')
expenses_local_detailed[expenses_local_detailed['category'] == 'Маркетплейсы'].to_csv(f'{FINAL_DIR}/consumption_market.csv')

## Дороги

In [ ]:
connection = pd.read_parquet('../data/connection.parquet')
connection.head()

,territory_id_x,territory_id_y,distance,type
0,2,1,100.9,highway
1,3,1,36.1,highway
2,3,2,85.0,highway
3,4,3,44.4,highway
4,4,1,59.9,highway


In [ ]:
connection = connection[(connection['territory_id_x'].isin(t_id) & connection['territory_id_y'].isin(t_id))]

In [ ]:
connection2 = pd.read_excel(f'{DATA_DIR}/connection2.xlsx')

In [ ]:
connection = pd.concat([connection, connection2])

In [ ]:
import networkx as nx

# Функция расчёта времени по типу связи и расстоянию
def calculate_time(row):
    distance = row['distance']
    transport_type = row['type']
    
    if transport_type == 'highway':
        return distance / 60  # 60 км/ч
    elif transport_type == 'railway':
        return distance / 65  # 65 км/ч
    elif transport_type == 'air':
        return 2 + (distance / 500)  # 2 часа на логистику + 500 км/ч
    elif transport_type == 'water':
        return 4 + (distance / 30)   # 4 часа на логистику + 30 км/ч
    else:
        return float('inf')  # Неизвестный тип

# Добавляем столбец с временем
connection['time'] = connection.apply(calculate_time, axis=1)

# Создаём граф
G = nx.Graph()

# Добавляем рёбра в граф
for _, row in connection.iterrows():
    G.add_edge(row['territory_id_x'], row['territory_id_y'], weight=row['time'])

# Определяем хабы
hubs = [1953, 1969]  # Южно-Сахалинск, Холмск

# Рассчитываем минимальное время до ближайшего хаба для каждого муниципалитета
results = []
all_nodes = set(connection['territory_id_x']).union(set(connection['territory_id_y']))

print(f"Всего узлов в графе: {len(all_nodes)}")
print(f"Хабы: {hubs}")

for node in all_nodes:
    if node in hubs:
        # Для хабов время = 0
        results.append({'territory_id': node, 'min_time_to_hub': 0})
        continue
        
    min_time = float('inf')
    
    # Ищем минимальное время до любого хаба
    for hub in hubs:
        try:
            # Используем Дейкстру для поиска кратчайшего пути
            time_to_hub = nx.shortest_path_length(G, source=node, target=hub, weight='weight')
            min_time = min(min_time, time_to_hub)
        except nx.NetworkXNoPath:
            # Если пути нет, пропускаем этот хаб
            continue
    
    # Если нашли путь хотя бы до одного хаба
    if min_time != float('inf'):
        results.append({'territory_id': node, 'min_time_to_hub': min_time})
    else:
        # Если нет пути ни до одного хаба
        results.append({'territory_id': node, 'min_time_to_hub': None})
        print(f"Предупреждение: Для территории {node} не найден путь ни к одному хабу")

# Создаём DataFrame с результатами
result_df = pd.DataFrame(results)

# Сортируем по времени
result_df = result_df.sort_values('min_time_to_hub')

# Сохраняем результаты только с id
result_df[['territory_id', 'min_time_to_hub']].to_csv('transport_accessibility_index.csv', index=False)

print("\nРасчёт завершён! Результаты сохранены в transport_accessibility_index.csv")
print(f"Обработано территорий: {len(result_df)}")
print(f"Из них не имеют связи с хабами: {result_df['min_time_to_hub'].isna().sum()}")

print("\nТоп-10 самых доступных территорий:")
print(result_df.head(10).to_string(index=False))

print("\nТоп-10 самых изолированных территорий:")
print(result_df[result_df['min_time_to_hub'].notna()].tail(10).to_string(index=False))

Всего узлов в графе: 26
Хабы: [1953, 1969]

Расчёт завершён! Результаты сохранены в transport_accessibility_index.csv
Обработано территорий: 26
Из них не имеют связи с хабами: 0

Топ-10 самых доступных территорий:
 territory_id  min_time_to_hub
         1953         0.000000
         1969         0.000000
         1957         0.600000
         1955         0.635000
         1956         0.661538
         1958         0.718333
         1960         0.723077
         1970         0.803333
         1966         1.384615
         1968         2.498000

Топ-10 самых изолированных территорий:
 territory_id  min_time_to_hub
         1967         3.818000
          775         3.904000
          872         4.374000
         1963         4.430769
          608         4.636000
         1965         5.553846
          666         9.020000
         1597        10.290000
         3008        15.282000
         1964        21.518333


In [ ]:
districts_name = pd.read_excel(f'{DATA_DIR}/districts.xlsx')

In [ ]:
connection_class = result_df.merge(districts_name[['territory_id', 'municipal_district_name']], how='left', on='territory_id')

In [ ]:
connection_class = connection_class[connection_class['territory_id'].isin(t_id)]

In [ ]:
connection_class.describe()

,territory_id,min_time_to_hub
count,15.000000,15.000000
mean,1961.000000,2.201185
std,5.277987,1.749531
min,1953.000000,0.000000
25%,1956.500000,0.648269
50%,1961.000000,2.498000
75%,1965.500000,3.374462
max,1969.000000,5.553846


In [ ]:
def classify_accessibility_sakhalin(time):
    if time == 0:
        return "Хаб (центр)"
    elif time <= 1:
        return "Высокая доступность (<1 часа)"
    elif time <= 2.5:  # Медиана
        return "Средняя доступность (1-2.5 часа)"
    elif time <= 4:
        return "Низкая доступность (2.5-4 часа)"
    else:
        return "Крайняя изоляция (>4 часов)"

connection_class['accessibility_category'] = connection_class['min_time_to_hub'].apply(classify_accessibility_sakhalin)

In [ ]:
connection_class['accessibility_category'].value_counts()

accessibility_category
Низкая доступность (2.5-4 часа)     5
Высокая доступность (<1 часа)       4
Хаб (центр)                         2
Средняя доступность (1-2.5 часа)    2
Крайняя изоляция (>4 часов)         2
Name: count, dtype: int64

In [ ]:
population = pd.read_csv(f'{FINAL_DIR}/territory_population.csv')

In [ ]:
connection_class_population = connection_class.merge(population, how='left', on='territory_id')

In [ ]:
connection_class_population.pivot_table(index='accessibility_category', values='value', aggfunc='sum').reset_index()

,accessibility_category,value
0,Высокая доступность (<1 часа),97894
1,Крайняя изоляция (>4 часов),31614
2,Низкая доступность (2.5-4 часа),63290
3,Средняя доступность (1-2.5 часа),24197
4,Хаб (центр),219781


In [ ]:
connection_class_population.to_csv(f'{FINAL_DIR}/connection_class_population.csv')

In [ ]:
salary = pd.read_parquet(f'{DATA_DIR}/salary.parquet')

In [ ]:
salary

,territory_id,year,period,okved_name,okved_letter,value
0,3,2023,январь-декабрь,Все отрасли,0,46265.2
1,3,2023,январь-декабрь,Обрабатывающие производства,C,50330.1
2,3,2023,январь-декабрь,Услуги ЖКХ,D,43988.1
3,3,2023,январь-декабрь,Строительство,F,78478.4
4,3,2023,январь-декабрь,Торговля,G,39262.9
...,...,...,...,...,...,...
369807,2617,2024,январь-сентябрь,Гостиницы и общепит,I,NaN
369808,2617,2024,январь-сентябрь,Гос. управление и военн. безопасность,O,NaN
369809,2617,2024,январь-сентябрь,Образование,P,44966.9
369810,2617,2024,январь-сентябрь,Спорт и досуг,R,45781.5


In [ ]:
connection_class_salary = connection_class.merge(salary, how='left', on='territory_id')

In [ ]:
connection_class_salary.to_csv(f'{FINAL_DIR}/connection_class_salaty.csv')

## Миграция

In [ ]:
migration = pd.read_parquet(f'{DATA_DIR}/migration.parquet')

In [ ]:
migration

,territory_id,year,period,age,gender,value
0,2335,2023,год,75-79,Женщины,5.0
1,2335,2023,год,75-79,Мужчины,NaN
2,2335,2023,год,55-59,Женщины,-4.0
3,2335,2023,год,55-59,Мужчины,-4.0
4,2335,2023,год,15-19,Женщины,-11.0
...,...,...,...,...,...,...
106219,2334,2023,год,70-74,Мужчины,-3.0
106220,2334,2023,год,45-49,Женщины,3.0
106221,2334,2023,год,45-49,Мужчины,3.0
106222,2334,2023,год,85-89,Женщины,-1.0


In [ ]:
migration.describe()

,territory_id,year,value
count,106224.000000,106224.0,89551.000000
mean,1338.046939,2023.0,6.157039
std,773.004607,0.0,132.907773
min,1.000000,2023.0,-2730.000000
25%,668.000000,2023.0,-6.000000
50%,1322.000000,2023.0,-1.000000
75%,2005.000000,2023.0,5.000000
max,3014.000000,2023.0,10245.000000


In [ ]:
migration['age'].unique()

array(['75-79', '55-59', '15-19', '90-94', '50-54', '35-39', '95-99',
       '40-44', '0-4', '5-9', '65-69', '80-84', '30-34', '25-29', '3-5',
       '20-24', 'Всего', '10-14', '60-64', '70-74', '45-49', '85-89',
       '100'], dtype=object)

In [ ]:
AGE_GROUPS = ['10-14', '15-19', '20-24']

In [ ]:
migration[(migration['age'].isin(AGE_GROUPS)) & (migration['territory_id'].isin(t_id))].pivot_table(
    index=['territory_id'],
    values='value',
    aggfunc='sum'
).reset_index().merge(districts, how='left', on='territory_id')

,territory_id,value,municipal_district_name,municipal_district_center,municipal_district_center_lat,municipal_district_center_lon
0,1953,-164.0,городской округ город Южно-Сахалинск,г Южно-Сахалинск,46.957427,142.727438
1,1954,38.0,городской округ Александровск-Сахалинский район,г Александровск-Сахалинский,50.897498,142.159167
2,1955,50.0,городской округ Анивский,г Анива,46.714777,142.527515
3,1956,144.0,городской округ Долинский,г Долинск,47.324146,142.797936
4,1957,52.0,городской округ Корсаковский,г Корсаков,46.632815,142.781015
5,1959,-46.0,городской округ Макаровский,г Макаров,48.624253,142.781400
6,1960,44.0,городской округ Невельский,г Невельск,46.687780,141.860114
7,1961,-94.0,городской округ Ногликский,пгт Ноглики,51.795362,143.136508
8,1962,-70.0,городской округ Охинский,г Оха,53.589199,142.949684
9,1963,26.0,городской округ Поронайский,г Поронайск,49.221637,143.101576


### Население для расчета показателей по вкладам и кредитам

In [18]:
districts_name = pd.read_excel(f'{DATA_DIR}/district_name.xlsx')
districts_name_dfo = districts_name[(districts_name['federal_districts_name_short'] == 'ДФО') & (districts_name['year_to'] == 9999)]

In [19]:
districts_name_dfo

,territory_id,oktmo,district_name_short,district_status,district_type,year_from,year_to,region_name,federal_districts_name,federal_districts_name_short
83,84,81-701-000-000,Улан-Удэ,административный центр,город,2018,9999,Республика Бурятия,Дальневосточный федеральный округ,ДФО
84,85,81-720-000-000,Северобайкальск,NaN,город,2018,9999,Республика Бурятия,Дальневосточный федеральный округ,ДФО
85,86,81-603-000-000,Баргузинский,NaN,село,2018,9999,Республика Бурятия,Дальневосточный федеральный округ,ДФО
86,87,81-606-000-000,Баунтовский,NaN,село,2018,9999,Республика Бурятия,Дальневосточный федеральный округ,ДФО
87,88,81-609-000-000,Бичурский,NaN,село,2018,9999,Республика Бурятия,Дальневосточный федеральный округ,ДФО
...,...,...,...,...,...,...,...,...,...,...
3070,805,05-553-000-000,Черниговский,NaN,село,2024,9999,Приморский край,Дальневосточный федеральный округ,ДФО
3071,807,05-557-000-000,Шкотовский,NaN,поселок городского типа,2024,9999,Приморский край,Дальневосточный федеральный округ,ДФО
3072,791,05-514-000-000,Красноармейский,NaN,село,2024,9999,Приморский край,Дальневосточный федеральный округ,ДФО
3073,854,08-534-000-000,Охотский,NaN,рабочий поселок,2024,9999,Хабаровский край,Дальневосточный федеральный округ,ДФО


In [8]:
need_man_age = ['16', '17', '18',
       '19', '20', '21', '22', '23', '24', '25', '26', '27', '28',
       '29', '30', '31', '32', '33', '34', '35', '36', '37', '38',
       '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '52', '53', '54', '55', '56', '57', '58',
       '59']

need_woman_age = ['16', '17', '18',
       '19', '20', '21', '22', '23', '24', '25', '26', '27', '28',
       '29', '30', '31', '32', '33', '34', '35', '36', '37', '38',
       '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '52', '53', '54']

In [ ]:
population_full.loc[(population_full['age'].isin(need_man_age)) & (population_full['gender'] == 'Мужчины'), 'value'].sum() + \
population_full.loc[(population_full['age'].isin(need_woman_age)) & (population_full['gender'] == 'Женщины'), 'value'].sum()

254950.0

In [ ]:
population_full

,territory_id,age,gender,value
0,1960,0,Женщины,68.0
1,1960,0,Мужчины,74.0
2,1960,1,Женщины,69.0
3,1960,1,Мужчины,75.0
4,1960,10,Женщины,82.0
...,...,...,...,...
2587,1953,67,Женщины,1236.0
2588,1953,68,Женщины,1277.0
2589,1953,69,Женщины,1277.0
2590,1953,70+,Женщины,12662.0


In [4]:
population = pd.read_parquet(f'{DATA_DIR}/population.parquet')
population_2024 = population[population['year'] == 2024]

In [12]:
population_work_age = population_2024[
    ((population_2024['age'].isin(need_man_age)) & (population_2024['gender'] == 'Мужчины')) | 
    ((population_2024['age'].isin(need_woman_age)) & (population_2024['gender'] == 'Женщины'))].pivot_table(index='territory_id', values='value', aggfunc='sum').reset_index()

In [21]:
districts_name_dfo.merge(population_work_age, how='left', on='territory_id').pivot_table(index='region_name', values='value', aggfunc='sum').reset_index()

,region_name,value
0,Амурская область,146100.0
1,Еврейская автономная область,80502.0
2,Забайкальский край,347388.0
3,Камчатский край,151722.0
4,Магаданская область,78659.0
5,Приморский край,994492.0
6,Республика Бурятия,0.0
7,Республика Саха (Якутия),566607.0
8,Сахалинская область,145271.0
9,Хабаровский край,720334.0


## Продукты

In [5]:
products = pd.read_excel(f'{DATA_DIR}/product_price_excel.xlsx')

In [6]:
products = products[['region', 'product', 'value', 'min', 'max']]

In [8]:
products['region'].str.strip().unique()

array(['Российская Федерация без учета новых субъектов (с 01.01.2023)',
       'Дальневосточный федеральный округ (с 03.11.2018)',
       'Приморский край', 'Хабаровский край', 'Сахалинская область',
       'Южно-Сахалинск', 'Корсаков', 'Оха', 'Поронайск'], dtype=object)

In [9]:
products['region'] = products['region'].str.strip()

In [10]:
products.loc[products['region'] == 'Российская Федерация без учета новых субъектов (с 01.01.2023)', 'region'] = 'Российская Федерация'
products.loc[products['region'] == 'Дальневосточный федеральный округ (с 03.11.2018)', 'region'] = 'Дальневосточный федеральный округ'

In [11]:
OTHER_REGIONS = ['Российская Федерация', 'Дальневосточный федеральный округ', 'Приморский край', 'Хабаровский край']
TARGET_REGION = 'Сахалинская область'
INSIDE_REGIONS = ['Южно-Сахалинск', 'Корсаков', 'Оха', 'Поронайск']

In [12]:
products_list = pd.read_excel(f'{DATA_DIR}/products_list.xlsx')

In [13]:
products_list

,category,product
0,"Мясо, колбасы и полуфабрикаты","Фарш мясной, кг"
1,"Мясо, колбасы и полуфабрикаты","Пельмени, манты, равиоли, кг"
2,"Мясо, колбасы и полуфабрикаты","Говядина (кроме бескостного мяса), кг"
3,"Мясо, колбасы и полуфабрикаты","Говядина бескостная, кг"
4,"Мясо, колбасы и полуфабрикаты","Свинина (кроме бескостного мяса), кг"
...,...,...
185,Молочные продукты,"Творог, кг"
186,Детское питание,"Творожок детский, кг"
187,Медикаменты,"Умифеновир (Арбидол), 100 мг, 10 капсул (табле..."
188,Хлебобулочные изделия,Хлеб и булочные изделия из пшеничной муки разл...


In [14]:
products = products.merge(products_list, how='inner', on='product')

In [15]:
products_region = products[products['region'] == TARGET_REGION].copy()
products_other = products[products['region'].isin(OTHER_REGIONS)].copy()

products_diff = products_other[['region', 'category', 'product', 'value']].merge(
    products_region[['product', 'value']], 
    how='inner', 
    on='product', 
    suffixes=['_other', '_target'])

In [17]:
products_diff.isna().sum()

region          0
category        0
product         0
value_other     0
value_target    0
dtype: int64

In [18]:
products_diff

,region,category,product,value_other,value_target
0,Российская Федерация,"Мясо, колбасы и полуфабрикаты","Фарш мясной, кг",477.493333,548.941667
1,Российская Федерация,"Мясо, колбасы и полуфабрикаты","Пельмени, манты, равиоли, кг",396.564167,518.396667
2,Российская Федерация,"Мясо, колбасы и полуфабрикаты","Говядина (кроме бескостного мяса), кг",560.923333,656.315000
3,Российская Федерация,"Мясо, колбасы и полуфабрикаты","Говядина бескостная, кг",768.103333,809.075833
4,Российская Федерация,"Мясо, колбасы и полуфабрикаты","Свинина (кроме бескостного мяса), кг",350.464167,474.602500
...,...,...,...,...,...
749,Хабаровский край,Молочные продукты,"Творог, кг",568.110833,684.241667
750,Хабаровский край,Детское питание,"Творожок детский, кг",617.601667,1074.975000
751,Хабаровский край,Медикаменты,"Умифеновир (Арбидол), 100 мг, 10 капсул (табле...",293.923333,326.642500
752,Хабаровский край,Хлебобулочные изделия,Хлеб и булочные изделия из пшеничной муки разл...,139.751667,154.085000


In [19]:
products_diff['diff'] = products_diff['value_target'] - products_diff['value_other']
products_diff['diff_percent'] = products_diff['diff'] * 100 / products_diff['value_other']

In [22]:
products_diff.describe()

,value_other,value_target,diff,diff_percent
count,754.000000,754.000000,754.000000,754.000000
mean,1380.570116,1466.714995,86.144879,19.263480
std,6406.341926,5915.105420,1359.602285,29.039732
min,3.486667,4.590000,-19709.071667,-69.011142
25%,110.254375,113.983333,2.548125,1.822028
50%,325.420000,371.304167,27.153750,15.289522
75%,681.730625,809.075833,111.968958,31.106522
max,92378.831667,72669.760000,10840.867500,195.271607


In [23]:
for region in products_diff['region'].unique():
    print(f'Регион сравнения: {region}')
    print(f'\nНаибольшая разница в цене')
    top_pos = products_diff.loc[
        products_diff['region'] == region, 
        ['product', 'value_other', 'value_target', 'diff_percent']
    ].nlargest(n=10, columns=['diff_percent'])
    print(top_pos.to_string(index=False))

    print(f'\nНаименьшая разница в цене')
    top_neg = products_diff.loc[
        products_diff['region'] == region, 
        ['product', 'value_other', 'value_target', 'diff_percent']
    ].nsmallest(n=10, columns=['diff_percent'])
    print(top_neg.to_string(index=False))

    print('===========================\n')

Регион сравнения: Российская Федерация

Наибольшая разница в цене
                    product  value_other  value_target  diff_percent
       Творожок детский, кг   404.435833   1074.975000    165.796181
    Изготовление гроба, шт.  7586.487500  18427.355000    142.897059
   Парацетамол, 10 таблеток    20.423333     47.564167    132.891301
   Сухие супы в пакетах, кг   659.249167   1470.868333    123.112657
                 Йод, 10 мл    44.271667     96.166667    117.219441
     Ягоды замороженные, кг   504.112500   1057.403333    109.755428
    Бумага туалетная, рулон    24.742500     51.832500    109.487724
                   Уголь, т  4159.489167   8525.499167    104.965053
Соль поваренная пищевая, кг    19.707500     39.598333    100.930272
                 Яблоки, кг   143.524167    280.314167     95.307991

Наименьшая разница в цене
                                                                 product  value_other  value_target  diff_percent
Полет в салоне экономического клас

In [51]:
inside_products = [
    'Смеси сухие молочные для детского питания, кг',
    'Яйца куриные, 10 шт.',
    'Молоко питьевое цельное стерилизованное 2,5-3,2% жирности, л',
    'Молоко питьевое цельное пастеризованное 2,5-3,2% жирности, л',
    'Сметана, кг',
    'Творог, кг',
    'Рыба живая и охлажденная, кг',
    'Фарш мясной, кг',
    'Куры охлажденные и мороженые, кг',
    'Колбаса вареная, кг',
    'Огурцы свежие, кг',
    'Помидоры свежие, кг',
    'Икра лососевых рыб, отечественная, кг',
    'Филе рыбное, кг',
    'Картофель, кг',
    'Рыба мороженая неразделанная, кг',
    'Говядина бескостная, кг'
]

In [52]:
products_inside = products_diff[products_diff['product'].isin(inside_products)]

In [53]:
products_inside.to_csv(f'{FINAL_DIR}/products_price.csv', index=False)

In [50]:
products_inside[products_inside['product'] == 'Молоко питьевое цельное пастеризованное 2,5-3,2% жирности, л']

,region,category,product,value_other,value_target,diff,diff_percent


In [44]:
products_final = products[(products['region'] == TARGET_REGION) & (products['product'].isin(inside_products))][['region', 'category', 'product', 'value']]

In [45]:
pd.concat([products_inside.rename(columns={'value_other': 'value'})[products_final.columns], products_final])

,region,category,product,value
0,Российская Федерация,"Мясо, колбасы и полуфабрикаты","Фарш мясной, кг",477.493333
5,Российская Федерация,"Мясо, колбасы и полуфабрикаты","Куры охлажденные и мороженые, кг",219.226667
10,Российская Федерация,"Мясо, колбасы и полуфабрикаты","Колбаса вареная, кг",532.237500
18,Российская Федерация,Морепродукты,"Филе рыбное, кг",550.244167
22,Российская Федерация,Морепродукты,"Икра лососевых рыб, отечественная, кг",6716.369167
33,Российская Федерация,Молочные продукты,"Молоко питьевое цельное пастеризованное 2,5-3,...",80.670833
35,Российская Федерация,Детское питание,"Смеси сухие молочные для детского питания, кг",1231.891667
44,Российская Федерация,Животного происхождения,"Яйца куриные, 10 шт.",115.655833
79,Российская Федерация,Овощи и фрукты,"Картофель, кг",43.996667
86,Российская Федерация,Овощи и фрукты,"Огурцы свежие, кг",179.679167


In [46]:
products_inside

,region,category,product,value_other,value_target,diff,diff_percent
0,Российская Федерация,"Мясо, колбасы и полуфабрикаты","Фарш мясной, кг",477.493333,548.941667,71.448333,14.963211
5,Российская Федерация,"Мясо, колбасы и полуфабрикаты","Куры охлажденные и мороженые, кг",219.226667,340.657500,121.430833,55.390539
10,Российская Федерация,"Мясо, колбасы и полуфабрикаты","Колбаса вареная, кг",532.237500,870.392500,338.155000,63.534606
18,Российская Федерация,Морепродукты,"Филе рыбное, кг",550.244167,495.866667,-54.377500,-9.882431
22,Российская Федерация,Морепродукты,"Икра лососевых рыб, отечественная, кг",6716.369167,5635.132500,-1081.236667,-16.098529
33,Российская Федерация,Молочные продукты,"Молоко питьевое цельное пастеризованное 2,5-3,...",80.670833,113.983333,33.312500,41.294355
35,Российская Федерация,Детское питание,"Смеси сухие молочные для детского питания, кг",1231.891667,1914.246667,682.355000,55.390828
44,Российская Федерация,Животного происхождения,"Яйца куриные, 10 шт.",115.655833,167.744167,52.088333,45.037359
79,Российская Федерация,Овощи и фрукты,"Картофель, кг",43.996667,80.839167,36.842500,83.739298
86,Российская Федерация,Овощи и фрукты,"Огурцы свежие, кг",179.679167,247.306667,67.627500,37.637919


### Цены для калькулятора

In [4]:
prices = pd.read_excel(f'{DATA_DIR}/price_calc.xlsx')

In [12]:
month_dict = {
    'январь': 1, 'января': 1,
    'февраль': 2, 'февраля': 2,
    'март': 3, 'марта': 3,
    'апрель': 4, 'апреля': 4,
    'май': 5, 'мая': 5,
    'июнь': 6, 'июня': 6,
    'июль': 7, 'июля': 7,
    'август': 8, 'августа': 8,
    'сентябрь': 9, 'сентября': 9,
    'октябрь': 10, 'октября': 10,
    'ноябрь': 11, 'ноября': 11,
    'декабрь': 12, 'декабря': 12
}

# Функция для преобразования
def russian_month_to_date(year_col, month_col):
    """Преобразует год и русское название месяца в дату"""
    return pd.to_datetime(
        year_col.astype(str) + '-' + 
        month_col.str.lower().map(month_dict).astype(str) + '-01'
    )

prices['date'] = russian_month_to_date(prices['year'], prices['month'])

In [14]:
prices.to_csv(f'{FINAL_DIR}/prices.csv')